### 批量模型推理

In [3]:
import os
import subprocess

def batch_inference_detector(script_path, config_template, checkpoint_template, image_folder, output_folder, dataset_sizes, device="cuda:0", batch_size=8, num_workers=4, use_fp16=True, score_thr=0.4, nms_thr=0.3, max_det=100, uncertainty_methods="sor", vis_scale=1.0):
    """
    批量运行 sual.inference.detector 模块。

    :param script_path: 模块的路径
    :param config_template: 配置文件路径模板，包含 {num} 占位符
    :param checkpoint_template: 检查点文件路径模板，包含 {num} 占位符
    :param image_folder: 图像文件夹路径
    :param output_folder: 输出文件夹路径模板，包含 {num} 占位符
    :param dataset_sizes: 数据集规模列表
    :param device: 设备类型 (如 "cuda:0")
    :param batch_size: 批量大小
    :param num_workers: 数据加载的线程数
    :param use_fp16: 是否使用 FP16
    :param score_thr: 置信度分数阈值
    :param nms_thr: NMS 阈值
    :param max_det: 最大检测数量
    :param uncertainty_methods: 不确定性度量方法
    :param vis_scale: 可视化比例
    """
    for num in dataset_sizes:
        config_path = config_template.format(num=num)
        checkpoint_path = checkpoint_template.format(num=num)
        output_path = os.path.join(output_folder.format(num=num))
        os.makedirs(output_path, exist_ok=True)

        # 检查配置文件和检查点是否存在
        if not os.path.exists(config_path):
            print(f"Config file not found: {config_path}. Skipping this dataset size.")
            continue
        if not os.path.exists(checkpoint_path):
            print(f"Checkpoint file not found: {checkpoint_path}. Skipping this dataset size.")
            continue

        command = [
            "python", "-m", script_path,
            config_path,
            checkpoint_path,
            image_folder,
            "--device", device,
            "--batch-size", str(batch_size),
            "--num-workers", str(num_workers),
            "--use-fp16" if use_fp16 else "",
            "--output-dir", output_path,
            "--score-thr", str(score_thr),
            "--nms-thr", str(nms_thr),
            "--max-det", str(max_det),
            "--uncertainty-methods", uncertainty_methods,
            "--vis-scale", str(vis_scale)
        ]

        # 移除空字符串参数
        command = [arg for arg in command if arg]

        print(f"Running: {' '.join(command)}")
        try:
            subprocess.run(command, check=True)
        except subprocess.CalledProcessError as e:
            print(f"Error running command for dataset size {num}: {e}")
            continue  # 跳过当前任务，继续下一个任务

# 示例用法
script_path = "sual.inference.detector"
config_template = "work_dirs/faster-rcnn_coco_{num}/faster-rcnn_coco_{num}.py"
checkpoint_template = "work_dirs/faster-rcnn_coco_{num}/epoch_20.pth"
image_folder = "./data/Eucalyptus_canopy/test2017"
output_folder = "test/cannoy_{num}"

dataset_sizes = [100, 200, 300, 500, 700, 900, 1100, 1500, 1700]

batch_inference_detector(
    script_path=script_path,
    config_template=config_template,
    checkpoint_template=checkpoint_template,
    image_folder=image_folder,
    output_folder=output_folder,
    dataset_sizes=dataset_sizes,
    device="cuda:0",
    batch_size=8,
    num_workers=4,
    use_fp16=True,
    score_thr=0.01,
    nms_thr=0.01,
    max_det=300,
    uncertainty_methods="sor",
    vis_scale=1.0
)


Running: python -m sual.inference.detector work_dirs/faster-rcnn_coco_100/faster-rcnn_coco_100.py work_dirs/faster-rcnn_coco_100/epoch_20.pth ./data/Eucalyptus_canopy/test2017 --device cuda:0 --batch-size 8 --num-workers 4 --use-fp16 --output-dir test/cannoy_100 --score-thr 0.01 --nms-thr 0.01 --max-det 300 --uncertainty-methods sor --vis-scale 1.0
Running: python -m sual.inference.detector work_dirs/faster-rcnn_coco_200/faster-rcnn_coco_200.py work_dirs/faster-rcnn_coco_200/epoch_20.pth ./data/Eucalyptus_canopy/test2017 --device cuda:0 --batch-size 8 --num-workers 4 --use-fp16 --output-dir test/cannoy_200 --score-thr 0.01 --nms-thr 0.01 --max-det 300 --uncertainty-methods sor --vis-scale 1.0
Running: python -m sual.inference.detector work_dirs/faster-rcnn_coco_300/faster-rcnn_coco_300.py work_dirs/faster-rcnn_coco_300/epoch_20.pth ./data/Eucalyptus_canopy/test2017 --device cuda:0 --batch-size 8 --num-workers 4 --use-fp16 --output-dir test/cannoy_300 --score-thr 0.01 --nms-thr 0.01 --m